In [1]:
!yes | pip uninstall torchvison
!pip install -qU torchvision

yes: standard output: Broken pipe
yes: write error
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.5.1 requires torch<1.10,>=1.7.0, but you have torch 1.10.1 which is incompatible.


In [2]:
import sys
import os

In [3]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/rain-au-pytorch-notebook'

role = sagemaker.get_execution_role()

print(f"Bucket : {bucket}/{prefix} \n Role : {role}")

Bucket : sagemaker-eu-west-3-543553163241/sagemaker/rain-au-pytorch-notebook 
 Role : arn:aws:iam::543553163241:role/service-role/AmazonSageMaker-ExecutionRole-20220116T155750


In [4]:
BASE_DIR = "pipelines/rain/"

In [149]:
base_job_prefix="RainAu"

processing_instance_count = 1
processing_instance_type = "ml.m5.xlarge"
training_instance_type = "ml.m5.xlarge"
inference_instance_type = "ml.m5.xlarge"
inference_instance_count = 1
model_approval_status = "PendingManualApproval"

# Getting and preprocessing the data
Data is stored in an s3 bucket in the account, needs to be processed by an SKlearn processor.

First let us update the git repository

In [53]:
!git pull origin main

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0
Unpacking objects: 100% (5/5), 462 bytes | 231.00 KiB/s, done.
From https://github.com/gmguarino/sagemaker-pipeline-rain-australia-build
 * branch            main       -> FETCH_HEAD
   dd602bc..69d1482  main       -> origin/main
Updating dd602bc..69d1482
Fast-forward
 pipelines/rain/evaluate.py | 10 +++++-----
 1 file changed, 5 insertions(+), 5 deletions(-)


In [7]:
input_data_source = "s3://rain-data-17012022/data/weatherAUS.csv"

In [8]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput


Examine the preprocessing script

In [9]:
!pygmentize pipelines/rain/preprocess.py

import logging
import sys

import argparse
import os
import pathlib
import pickle

import boto3
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))


def merge_two_dicts(x, y):
    """Merges two dicts, returning a new copy."""
    z = x.copy()
    z.update(y)
    return z


def cyclical_encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data


if __name__ == "__main__":

    logger.debug("Starting preprocessing.")
    # parser = argparse.ArgumentParser()
    # parser.add_argument("--input-data", type=str, re

In [10]:
sklearn_processor = SKLearnProcessor(
    framework_version='0.20.0',
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    volume_size_in_gb=15,
    base_job_name=f"rainau-notebook-preprocess",
    sagemaker_session=sagemaker_session,
    role=role,
)

sklearn_processor.run(code=os.path.join(BASE_DIR, "preprocess.py"),
    inputs=[ProcessingInput(source=input_data_source,
        destination='/opt/ml/processing/input/')],
    outputs=[
        ProcessingOutput(output_name="train",
            source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation",
            source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test",
            source="/opt/ml/processing/test"),
        ProcessingOutput(output_name="scaler",
            source="/opt/ml/processing/preprocess")
    ]
)


Job Name:  rainau-notebook-preprocess-2022-01-22-10-45-38-705
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://rain-data-17012022/data/weatherAUS.csv', 'LocalPath': '/opt/ml/processing/input/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-3-543553163241/rainau-notebook-preprocess-2022-01-22-10-45-38-705/input/code/preprocess.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-eu-west-3-543553163241/rainau-notebook-preprocess-2022-01-22-10-45-38-705/output/train', 'LocalPath': '/opt/ml/processing/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'validation', 'AppManaged

In [11]:
model_path = f"s3://{sagemaker_session.default_bucket()}/{base_job_prefix}/model"

In [12]:
model_path

's3://sagemaker-eu-west-3-543553163241/RainAu/model'

In [13]:
preprocessing_job_description = sklearn_processor.jobs[-1].describe()
output_config = preprocessing_job_description["ProcessingOutputConfig"]
processing_output_uris = {}
for output in output_config["Outputs"]:
    processing_output_uris[output["OutputName"]] = output["S3Output"]["S3Uri"]
print(processing_output_uris)

{'train': 's3://sagemaker-eu-west-3-543553163241/rainau-notebook-preprocess-2022-01-22-10-45-38-705/output/train', 'validation': 's3://sagemaker-eu-west-3-543553163241/rainau-notebook-preprocess-2022-01-22-10-45-38-705/output/validation', 'test': 's3://sagemaker-eu-west-3-543553163241/rainau-notebook-preprocess-2022-01-22-10-45-38-705/output/test', 'scaler': 's3://sagemaker-eu-west-3-543553163241/rainau-notebook-preprocess-2022-01-22-10-45-38-705/output/scaler'}


In [14]:
from sagemaker.pytorch import PyTorch

# Train the model in a Pytorch container

The script `train.py` takes care of the training and saving of the neural network

In [93]:
!git pull origin main

From https://github.com/gmguarino/sagemaker-pipeline-rain-australia-build
 * branch            main       -> FETCH_HEAD
Already up to date.


In [94]:
!pygmentize pipelines/rain/train.py

import argparse
import json
import logging
import os
import sys
import shutil
import tarfile
import pathlib

import pandas as pd
import numpy as np

#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import Dataset

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))


# Dataset class to handle data loading
class RainDataset(Dataset):
    def __init__(self, csv_file, root_dir):
        self.data = pd.read_csv(os.path.join(root_dir, csv_file))
        self.labels = np.asarray(self.data.iloc[:, 0])
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        single_label = self.labels[idx]

        data_array = np.asarray(self.data.i

    parser.add_argument("--model-dir", type=str, default=os.environ["SM_MODEL_DIR"])
    parser.add_argument("--data-dir", type=str, default=os.environ["SM_CHANNEL_TRAIN"])
    parser.add_argument("--num-gpus", type=int, default=os.environ["SM_NUM_GPUS"])

    train(parser.parse_args())

    # TODO: REMOVE JIT LEAVE AS NORMAL MODEL


In [95]:
pytorch_estimator = PyTorch(os.path.join(BASE_DIR, 'train.py'),
    instance_type=training_instance_type,
    instance_count=1,
    framework_version='1.8.0',
    py_version='py3',
    base_job_name=f"{base_job_prefix}-torch-train-notebook",
    output_path=model_path,
    hyperparameters={'epochs': 1, 'batch-size': 32, 'lr': 0.00009},
    role=role)
pytorch_estimator.fit({'train': processing_output_uris["train"]})

2022-01-22 17:59:09 Starting - Starting the training job...
2022-01-22 17:59:11 Starting - Launching requested ML instancesProfilerReport-1642874349: InProgress
...
2022-01-22 18:00:01 Starting - Preparing the instances for training.........
2022-01-22 18:01:33 Downloading - Downloading input data...
2022-01-22 18:01:54 Training - Downloading the training image.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-01-22 18:02:12,745 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-01-22 18:02:12,747 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-01-22 18:02:12,757 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-01-22 18:02:15,775 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-01-22 18:02:16,103 sagemaker-training-toolkit INFO     No GPUs detected (normal if 

Train Epoch: 1 [60800/89275 (68%)] Loss: 0.561671
Train Epoch: 1 [64000/89275 (72%)] Loss: 0.621409
Train Epoch: 1 [67200/89275 (75%)] Loss: 0.432216
Train Epoch: 1 [70400/89275 (79%)] Loss: 0.330699
Train Epoch: 1 [73600/89275 (82%)] Loss: 0.453471
Train Epoch: 1 [76800/89275 (86%)] Loss: 0.448097
Train Epoch: 1 [80000/89275 (90%)] Loss: 0.439875
Train Epoch: 1 [83200/89275 (93%)] Loss: 0.371524
Train Epoch: 1 [86400/89275 (97%)] Loss: 0.578737
Saving the model.
/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
INFO:__main__:Train Epoch: 1 [3200/89275 (4%)] Loss: 0.712374
INFO:__main__:Train Epoch: 1 [6400/89275 (7%)] Loss: 0.694392
INFO:__main__:Train Epoch: 1 [9600/89275 (11%)] Loss: 0.664840
INFO:__main__:Train Epoch: 1 [12800/89275 (14%)] Loss: 0.637017
INFO:__main__:Train Epoch: 1 [16000/89275 (18%)] Loss

In [96]:
pytorch_estimator.model_data

's3://sagemaker-eu-west-3-543553163241/RainAu/model/RainAu-torch-train-notebook-2022-01-22-17-59-09-214/output/model.tar.gz'

# Evaluate the model

In [97]:
!git pull origin main

From https://github.com/gmguarino/sagemaker-pipeline-rain-australia-build
 * branch            main       -> FETCH_HEAD
Already up to date.


In [98]:
from sagemaker.processing import FrameworkProcessor
from sagemaker.workflow.properties import PropertyFile

In [99]:
pytorch_processor = FrameworkProcessor(
    PyTorch,
    instance_type=processing_instance_type,
    instance_count=1,
    framework_version='1.8.0',
    base_job_name=f"{base_job_prefix}-torch-eval",
    sagemaker_session=sagemaker_session,
    role=role,
    command=["python3"],
)

pytorch_processor.run(code=os.path.join(BASE_DIR, "evaluate.py"),
    inputs=[ProcessingInput(source=pytorch_estimator.model_data, destination="/opt/ml/processing/model"), 
        # ProcessingInput(source=model_path,destination="/opt/ml/processing/model"), 
        ProcessingInput(source=processing_output_uris["validation"],
            destination="/opt/ml/processing/validation")
    ],
    outputs=[ProcessingOutput(output_name="evaluation",source="/opt/ml/processing/evaluation")]
)


Job Name:  RainAu-torch-eval-2022-01-22-18-03-22-287
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-3-543553163241/RainAu/model/RainAu-torch-train-notebook-2022-01-22-17-59-09-214/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/model', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-3-543553163241/rainau-notebook-preprocess-2022-01-22-10-45-38-705/output/validation', 'LocalPath': '/opt/ml/processing/validation', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-3-543553163241/RainAu-torch-eval-2022-01-22-18-03-22-287/source/sourcedir.tar.gz', 'LocalPath': '/opt/ml/processing/input/code/', 'S3DataType

In [100]:
evaluation_job_description = pytorch_processor.jobs[-1].describe()
output_config = evaluation_job_description["ProcessingOutputConfig"]
evaluation_output_uris = {}
for output in output_config["Outputs"]:
    evaluation_output_uris[output["OutputName"]] = output["S3Output"]["S3Uri"]
print(evaluation_output_uris)

{'evaluation': 's3://sagemaker-eu-west-3-543553163241/RainAu-torch-eval-2022-01-22-18-03-22-287/output/evaluation'}


In [101]:
import json
import boto3

s3_client = boto3.client('s3')

uri = evaluation_output_uris["evaluation"]+ "/evaluation.json"

obj = s3_client.get_object(Bucket=uri.split("/")[2], Key="/".join(uri.split("/")[3:]))

report = json.loads(obj['Body'].read())
#df = pd.read_csv(io.BytesIO(obj['Body'].read()))


#with open(evaluation_output_uris["evaluation"]+ "/evaluation.json") as jf:
#    report = json.load(jf)
    
print(report)

{'regression_metrics': {'accuracy': 0.8328653846153845, 'precision': 0.6509762873908326, 'recall': 0.4358036483979058, 'f1_score': 0.5214194391645994}}


Here we have only trained the model for one epoch (demo purposes), so the performance is not great. Let's say though that we are satisfied, we can then decide to deploy our model to an endpoint. Let's start by loading in a Pytorch model.

In [131]:
!git pull origin main

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0
Unpacking objects: 100% (5/5), 494 bytes | 247.00 KiB/s, done.
From https://github.com/gmguarino/sagemaker-pipeline-rain-australia-build
 * branch            main       -> FETCH_HEAD
   e9a4a7b..dff7c1d  main       -> origin/main
Updating e9a4a7b..dff7c1d
Fast-forward
 pipelines/rain/inference.py | 35 ++++++++++++++++++-----------------
 1 file changed, 18 insertions(+), 17 deletions(-)


In [132]:
from sagemaker.pytorch import PyTorchModel

In [133]:
pytorch_model_deployment = PyTorchModel(
    entry_point='inference.py',
    source_dir=BASE_DIR,
    model_data=pytorch_estimator.model_data,
    role=role,
    framework_version='1.8.0',
    py_version='py3',
    name="rain-au-deployment-model"
)


In [134]:
help(PyTorchModel)

Help on class PyTorchModel in module sagemaker.pytorch.model:

class PyTorchModel(sagemaker.model.FrameworkModel)
 |  PyTorchModel(model_data, role, entry_point, framework_version=None, py_version=None, image_uri=None, predictor_cls=<class 'sagemaker.pytorch.model.PyTorchPredictor'>, model_server_workers=None, **kwargs)
 |  
 |  An PyTorch SageMaker ``Model`` that can be deployed to a SageMaker ``Endpoint``.
 |  
 |  Method resolution order:
 |      PyTorchModel
 |      sagemaker.model.FrameworkModel
 |      sagemaker.model.Model
 |      sagemaker.model.ModelBase
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, model_data, role, entry_point, framework_version=None, py_version=None, image_uri=None, predictor_cls=<class 'sagemaker.pytorch.model.PyTorchPredictor'>, model_server_workers=None, **kwargs)
 |      Initialize a PyTorchModel.
 |      
 |      Args:
 |          model_data (str): The S3 location of a SageMaker model data
 |            

In [144]:
predictor = pytorch_model_deployment.deploy(
    initial_instance_count=inference_instance_count,
    instance_type=inference_instance_type,
    endpoint_name="pytorch-rain-au-model-test"
)

Using already existing model: rain-au-deployment-model


-----!

In [145]:
import pandas as pd
import io


uri = processing_output_uris["test"] + "/test.csv"
print(uri.split("/")[2], "/".join(uri.split("/")[3:]))

obj = s3_client.get_object(Bucket=uri.split("/")[2], Key="/".join(uri.split("/")[3:]))

test_data = pd.read_csv(io.BytesIO(obj['Body'].read()))
test_data.head()

sagemaker-eu-west-3-543553163241 rainau-notebook-preprocess-2022-01-22-10-45-38-705/output/test/test.csv


,RainTomorrow,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,year,month_sin,month_cos,day_sin,day_cos
0,1,0.928162,0.692446,-0.312621,-0.203581,1.264694,-2.063213,1.045228,-0.073333,0.885879,...,1.464068,1.087018,-0.029011,-0.593947,-0.529795,-1.879575,-1.244369,0.728636,0.416432,-1.334442
1,0,-1.601947,0.205466,-0.707636,-0.275097,-0.371139,1.236542,1.045228,-0.073333,-1.107113,...,1.025756,-1.760956,-0.230584,-0.623142,-0.529795,-1.485514,-0.725379,1.245139,-0.803968,1.199371
2,0,-0.899139,-0.312933,-0.552452,-0.275097,-0.119472,0.148710,0.832195,-0.530619,1.107323,...,0.149133,0.137693,-0.091034,-0.447975,-0.529795,1.272917,-1.244369,-0.682476,0.909858,-1.055520
3,0,0.436196,-1.428275,-0.242083,-0.275097,-0.119472,0.148710,-0.232974,-0.759262,-0.885669,...,0.149133,0.137693,-1.191932,-0.053851,-0.529795,0.878855,-1.244369,-0.682476,1.255292,-0.601030
4,0,-0.618016,-0.218678,-0.256190,-0.251258,-1.063222,-2.462085,-0.446007,-0.759262,1.771653,...,0.149133,0.137693,-0.618224,-0.842100,-0.529795,-0.303329,1.211519,-0.682476,-1.403140,-0.044639


In [146]:
record_n = 2

row = test_data.iloc[record_n]
y = int(row["RainTomorrow"])
X = row.drop("RainTomorrow").values
y, X

(0,
 array([-0.89913902, -0.31293265, -0.55245169, -0.27509664, -0.11947224,
         0.1487104 ,  0.8321948 , -0.53061881,  1.10732274,  0.48503533,
         0.33506294, -0.65044884, -1.16154379,  0.26606081,  0.81076972,
         0.80280735,  0.14913321,  0.13769321, -0.0910336 , -0.44797542,
        -0.52979545,  1.27291668, -1.24436937, -0.68247589,  0.90985766,
        -1.05551973]))

In [147]:
prediction = predictor.predict(X)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://eu-west-3.console.aws.amazon.com/cloudwatch/home?region=eu-west-3#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-rain-au-model-test in account 543553163241 for more information.

In [148]:
sagemaker_session.delete_endpoint(
    endpoint_name="pytorch-rain-au-model-test"
    # endpoint_name = predictor.endpoint_name
)